In [ ]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from tqdm import tqdm
from functions.estimators import MonteCarlo

### Creation of Parameter Grid for Training

In [ ]:
# Generate SABR parameters
def generate_sabr_parameters(n_samples):
    alpha = np.random.uniform(0.01, 0.1, n_samples)
    beta = np.full(n_samples, 0.5) 
    nu = np.random.uniform(0.1, 3.0, n_samples)
    rho = np.random.uniform(-1.0, 0.5, n_samples)

    return alpha, beta, rho, nu


# Generate Market Data
def generate_market_data(n_samples):
    maturities_days = {
    '30d': 30,
    '60d': 60,
    '90d': 90,
    '180d': 180,
    '270d': 270,
    '1y': 360,
    '2y': 2 * 360,
    '3y': 3 * 360,
    '4y': 4 * 360,
    '5y': 5 * 360,
    '6y': 6 * 360,
    '7y': 7 * 360,
    '8y': 8 * 360,
    '9y': 9 * 360,
    '10y': 10 * 360,
}

    # Sample from the values (in days), then convert to year fractions
    T_days = np.random.choice(list(maturities_days.values()), size=n_samples, replace=True)
    T = T_days / 360

    F0 = np.random.uniform(0.04, 0.06, n_samples)
    strikes = np.array([
        np.arange(f0 - 0.025, f0 + 0.025 + 1e-10, 0.00125)
        for f0 in F0
    ])
    
    return strikes, T, F0

### Generation Simulated Dataset

In [ ]:
mc = MonteCarlo()

def generate_simulated_dataset(n_samples):
    alpha, beta, rho, nu = generate_sabr_parameters(n_samples)
    strikes, T, F0 = generate_market_data(n_samples)

    dataset = []

    for i in tqdm(range(n_samples), desc="Simulating"):

        # Calculate implied vols
        call_prices, call_ivs = mc.calc_price_and_iv(alpha=alpha[i], beta=beta[i], rho=rho[i], nu=nu[i], K_vector=strikes[i], F0=F0[i], T=T[i], no_of_sim=2000000, no_of_steps=1400)
        
        # Append one row per strike
        for k in range(len(strikes[i])):
            dataset.append([
                alpha[i], beta[i], rho[i], nu[i], T[i], F0[i], strikes[i][k], call_ivs[k], call_prices[k]
            ])


    df = pd.DataFrame(dataset, columns=['alpha', 'beta', 'rho', 'nu', 'T', 'F0', 'K', 'IV', 'price'])
    return df

In [ ]:
df0 = generate_simulated_dataset(10000)
df0.to_csv("saved_test_dataframe0.csv", index=False)

Simulating:   0%|          | 0/2000 [00:00<?, ?it/s]

Simulating: 100%|██████████| 2000/2000 [1:54:12<00:00,  3.43s/it]
